Aidan Coyle, afcoyle@uw.edu

2021/01/20

Roberts lab at SAFS

## BLASTing DEGs against database of all Alveolata sequences

This script takes a newline-separated file of accession IDs for all DEGs and BLASTs it against all Alveolata nucleotide sequences. Alveolata is the superphylum containing all dinoflagellates, including _Hematodinium_. This should exclude all but the most highly-conserved _C. bairdi_ genes from the DEGs, allowing us to examine _Hematodinium_ DEGs on an individual basis.

All Alveolata nucleotide sequences downloaded from the NCBI Taxonomy Browser at https://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi as a FASTA file. Download was made at 00:06 on 2021-02-17

In [36]:
# Create blast database from all Alveolata nucleotide sequences
!makeblastdb \
-in ../data/alveolata_sequences.fasta \
-dbtype nucl \
-parse_seqids \
-out ../data/blast_db/alveolata_nucleotides_2021_02_22/alveolata_uniprot_2021_02



Building a new DB, current time: 02/22/2021 23:57:21
New DB name:   /mnt/c/Users/acoyl/Documents/GitHub/hemat_bairdii_transcriptome/data/blast_db/alveolata_nucleotides_2021_02_22/alveolata_uniprot_2021_02
New DB title:  ../data/alveolata_sequences.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Ignoring sequence 'lcl|MT078137.1_cds_QJQ82428.1_1' as it has no sequence data
Adding sequences from FASTA; added 1566546 sequences in 81.6705 seconds.


In [59]:
# Show what the first column of the DEG list looks like, used next
!cut -f1 < ../graphs/DESeq2_output/amb2_vs_elev2_indiv/DEGlist.txt | head -n 5

TRINITY_DN978_c2_g1_i3
TRINITY_DN29869_c0_g1_i18
TRINITY_DN135171_c0_g1_i12
TRINITY_DN311_c0_g1_i18
TRINITY_DN183158_c0_g1_i1
cut: write error: Broken pipe


In [62]:
# Select first column - transcript ID - of the DEG list we made earlier, 
# turn into temporary file
!cut -f1 < ../graphs/DESeq2_output/amb2_vs_elev2_indiv/DEGlist.txt \
> ../output/BLASTn/tempids.txt

In [64]:
# Cross-reference transcript IDs and our transcriptome to get sequences for DEGs
# Pull the line containing a match (the transcript ID) 
# and the one following (the fasta sequence), write to file
!grep -w -A 1 -Ff ../output/BLASTn/tempids.txt \
../data/cbai_hemat_transcriptome_v2.0.fasta --no-group-separator > \
../output/BLASTn/alveolata/amb2_vs_elev2_indiv_DEGs.fasta

In [67]:
# Check length of original DEG list file
!wc -l ../graphs/DESeq2_output/amb2_vs_elev2_indiv/DEGlist.txt

2067 ../graphs/DESeq2_output/amb2_vs_elev2_indiv/DEGlist.txt


In [68]:
# See how many FASTA sequences we have
!grep -c ">" ../output/BLASTn/alveolata/amb2_vs_elev2_indiv_DEGs.fasta

2067


In [69]:
# Looks good! Remove temporary file
!rm ../output/BLASTn/tempids.txt

In [102]:
# Blast all DEG sequences against our database of Alveolata sequences
!blastn \
-task="blastn" \
-query ../output/BLASTn/alveolata/amb2_vs_elev2_indiv_DEGs.fasta \
-db ../data/blast_db/alveolata_nucleotides_2021_02_22/alveolata_uniprot_2021_02 \
-out ../output/BLASTn/alveolata/amb2_vs_elev2.tab \
-max_target_seqs 1 \
-outfmt 6 \
-num_threads 4

In [84]:
# See how many matches we got 
!wc -l ../output/BLASTn/alveolata/amb2_vs_elev2.tab

43 ../output/BLASTn/alveolata/amb2_vs_elev2.tab


In [85]:
# Look at those matches
!head ../output/BLASTn/alveolata/amb2_vs_elev2.tab

TRINITY_DN17_c0_g1_i12	HBJU01001350.1_cds_1	75.340	515	117	8	809	1318	519	10	1.73e-60	239
TRINITY_DN61_c0_g1_i6	HBLK01034498.1_cds_CAE2923986.1_1	85.507	276	38	2	1321	1595	398	124	7.42e-75	287
TRINITY_DN8754_c1_g1_i1	HBLO01048875.1_cds_CAE3072170.1_1	73.904	456	108	9	268	719	656	208	1.29e-40	172
TRINITY_DN8756_c1_g1_i3	HBJK01013588.1_cds_CAE1088085.1_1	89.231	65	7	0	647	711	274	210	2.18e-13	82.4
TRINITY_DN43328_c0_g1_i10	HBLC01024351.1_cds_CAE2698820.1_1	100.000	32	0	0	631	662	277	308	1.20e-06	60.2
TRINITY_DN43328_c0_g1_i10	HBLC01024351.1_cds_CAE2698820.1_1	100.000	29	0	0	634	662	268	296	5.58e-05	54.7
TRINITY_DN431_c0_g2_i18	HBKY01019556.1_cds_1	72.336	1070	255	29	1730	2792	1103	68	6.00e-78	298
TRINITY_DN41764_c0_g1_i3	HBLO01010512.1_cds_CAE3048796.1_1	86.517	89	12	0	563	651	359	447	4.47e-18	99.0
TRINITY_DN883_c0_g1_i21	HBNJ01035065.1_cds_1	72.068	1407	356	34	147	1533	149	1538	1.28e-104	387
TRINITY_DN8273_c0_g1_i64	HBHF01001998.1_cds_1	100.000	28	0	0	20	47	297	270	4.49e-04	52.8


## Repeat the same process for Elevated Day 0 vs. Elevated Day 2

In [73]:
# Select first column - transcript ID - of the DEG list we made earlier, 
# turn into temporary file
!cut -f1 < ../graphs/DESeq2_output/elev0_vs_elev2_indiv/DEGlist.txt \
> ../output/BLASTn/tempids.txt

In [74]:
# Cross-reference transcript IDs and our transcriptome to get sequences for DEGs
# Pull the line containing a match (the transcript ID) 
# and the one following (the fasta sequence), write to file
!grep -w -A 1 -Ff ../output/BLASTn/tempids.txt \
../data/cbai_hemat_transcriptome_v2.0.fasta --no-group-separator > \
../output/BLASTn/alveolata/elev0_vs_elev2_indiv_DEGs.fasta

In [75]:
# Check length of original DEG list file
!wc -l ../graphs/DESeq2_output/elev0_vs_elev2_indiv/DEGlist.txt

338 ../graphs/DESeq2_output/elev0_vs_elev2_indiv/DEGlist.txt


In [76]:
# See how many FASTA sequences we have
!grep -c ">" ../output/BLASTn/alveolata/elev0_vs_elev2_indiv_DEGs.fasta

338


In [77]:
# Looks good! Remove temporary file
!rm ../output/BLASTn/tempids.txt

In [101]:
# Blast all DEG sequences against our database of Alveolata sequences
!blastn \
-task="blastn" \
-query ../output/BLASTn/alveolata/elev0_vs_elev2_indiv_DEGs.fasta \
-db ../data/blast_db/alveolata_nucleotides_2021_02_22/alveolata_uniprot_2021_02 \
-out ../output/BLASTn/alveolata/elev0_vs_elev2.tab \
-max_target_seqs 1 \
-outfmt 6 \
-num_threads 4

In [86]:
# See how many matches we got 
!wc -l ../output/BLASTn/alveolata/elev0_vs_elev2.tab

1 ../output/BLASTn/alveolata/elev0_vs_elev2.tab


In [87]:
# Look at that match
!cat ../output/BLASTn/alveolata/elev0_vs_elev2.tab

TRINITY_DN118828_c0_g1_i2	HBGB01029743.1_cds_1	87.500	56	7	0	353	408	316	371	1.08e-08	65.8


# BREAK: EXPERIMENTAL STUFF STARTING HERE


In [91]:
# See if we have any matches that correspond to genes
!wc -l ../output/TPM_counts/amb2_vs_elev2_DEG_TPMs.txt

295 ../output/TPM_counts/amb2_vs_elev2_DEG_TPMs.txt


In [90]:
# Get gene ID and TPM counts for our single elevated day 0 vs elevated day 2 hemat gene
!grep TRINITY_DN118828_c0_g1_i2 ../output/TPM_counts/elev0_vs_elev2_DEG_TPMs.txt

In [93]:
!wc -l ../data/cbai_hemat_diamond_blastx_table_transcriptome_v2.0.txt

147454 ../data/cbai_hemat_diamond_blastx_table_transcriptome_v2.0.txt


In [99]:
!grep TRINITY_DN118828 ../data/cbai_hemat_diamond_blastx_table_transcriptome_v2.0.txt